In [1]:
import pickle
import pandas as pd
import numpy as np
import torch
import transformers
import bitsandbytes as bnb
import networkx as nx

random_state = 1

In [2]:
def euclid(list1, list2):
    return sum((p-q)**2 for p, q in zip(list1, list2)) ** .5

In [3]:
with open('df_annotations_embeddings.pkl', 'rb') as f:
    df = pickle.load(f)
df

,Document,Split,Name,Id,Text,Type,Scheme,embeddings
0,A2008_Commission of the European Communities v...,1,prem,A1,must held first Article 4c CS prohibits granti...,L,Rule,"[-0.01507568359375, -1.73828125, 1.4287109375,..."
1,A2008_Commission of the European Communities v...,1,prem,A2,Also clear consistent caselaw Articles 4 CS 67...,L,"[Itpr, Prec, Rule]","[0.93017578125, -1.0146484375, 0.0575256347656..."
2,A2008_Commission of the European Communities v...,1,prem,A3,Court deduces Article 67 CS covers general mea...,L,"[Prec, Rule]","[0.195556640625, -1.734375, -0.1546630859375, ..."
3,A2008_Commission of the European Communities v...,1,prem,A4,Court also held action taken Article 67 CS can...,L,"[Prec, Rule]","[0.404296875, -0.2352294921875, 0.34326171875,..."
4,A2008_Commission of the European Communities v...,1,prem,A5,Court held particular inconceivable authors EC...,L,"[Itpr, Prec, Rule]","[1.0849609375, -0.98828125, 0.39453125, 0.8774..."
...,...,...,...,...,...,...,...,...
2530,R2021_World Duty Free v,5,prem,H7,question matter dialogue Spanish authorities C...,F,NaN,"[2.109375, -1.3642578125, -0.62744140625, 2.50..."
2531,R2021_World Duty Free v,5,prem,H8,present case action first instance WDFG compla...,F,NaN,"[0.880859375, -0.65966796875, -0.67626953125, ..."
2532,R2021_World Duty Free v,5,prem,H9,regard General Court recalled paragraph 212 ju...,F,NaN,"[2.0234375, -1.150390625, -0.70947265625, 1.59..."
2533,R2021_World Duty Free v,5,prem,H10,General Court err law concluding therefrom par...,F,NaN,"[1.6494140625, -1.5732421875, -0.857421875, 1...."


In [4]:
df_train = df[df['Split'] != 2]
df_train = df_train.dropna(subset = ['Scheme'])
df_train

,Document,Split,Name,Id,Text,Type,Scheme,embeddings
0,A2008_Commission of the European Communities v...,1,prem,A1,must held first Article 4c CS prohibits granti...,L,Rule,"[-0.01507568359375, -1.73828125, 1.4287109375,..."
1,A2008_Commission of the European Communities v...,1,prem,A2,Also clear consistent caselaw Articles 4 CS 67...,L,"[Itpr, Prec, Rule]","[0.93017578125, -1.0146484375, 0.0575256347656..."
2,A2008_Commission of the European Communities v...,1,prem,A3,Court deduces Article 67 CS covers general mea...,L,"[Prec, Rule]","[0.195556640625, -1.734375, -0.1546630859375, ..."
3,A2008_Commission of the European Communities v...,1,prem,A4,Court also held action taken Article 67 CS can...,L,"[Prec, Rule]","[0.404296875, -0.2352294921875, 0.34326171875,..."
4,A2008_Commission of the European Communities v...,1,prem,A5,Court held particular inconceivable authors EC...,L,"[Itpr, Prec, Rule]","[1.0849609375, -0.98828125, 0.39453125, 0.8774..."
...,...,...,...,...,...,...,...,...
2520,R2021_World Duty Free v,5,prem,G4,accordance settled caselaw appeal plea directe...,L,Prec,"[0.5693359375, -1.30078125, 0.0073127746582031..."
2521,R2021_World Duty Free v,5,prem,G5,present case even fifth part single ground app...,L,Prec,"[0.484130859375, -0.362548828125, -0.387695312..."
2526,R2021_World Duty Free v,5,prem,H4,pointed paragraph 146 appeal plea directed gro...,"[F, L]",Itpr,"[0.34423828125, -0.83349609375, -0.04077148437..."
2528,R2021_World Duty Free v,5,prem,H6,regard true decision adopts end examination Co...,L,Princ,"[1.3798828125, -0.8056640625, 0.01039886474609..."


In [5]:
df_test = df[df['Split'] == 2]
df_test = df_test.dropna(subset = ['Scheme'])
df_test

,Document,Split,Name,Id,Text,Type,Scheme,embeddings
172,A2010_NDSHT Nya Destination Stockholm Hotell &...,2,prem,A2,regard must observed follows Article 58 Statut...,L,"[Prec, Rule]","[0.5068359375, -1.6494140625, -0.43408203125, ..."
177,A2010_NDSHT Nya Destination Stockholm Hotell &...,2,prem,B2,regard Court repeatedly held action annulment ...,L,"[Prec, Rule]","[1.076171875, -0.91064453125, -0.53271484375, ..."
178,A2010_NDSHT Nya Destination Stockholm Hotell &...,2,prem,B3,follows also settled caselaw concerning admiss...,L,Prec,"[0.90185546875, -1.087890625, -0.5048828125, 1..."
179,A2010_NDSHT Nya Destination Stockholm Hotell &...,2,prem,B4,contrast form act decision adopted principle i...,L,Itpr,"[0.92431640625, -0.471435546875, -0.3063964843..."
180,A2010_NDSHT Nya Destination Stockholm Hotell &...,2,prem,B5,therefore principle irrelevant classification ...,L,Itpr,"[0.7216796875, -0.82373046875, 0.39501953125, ..."
...,...,...,...,...,...,...,...,...
2123,R2017_European Commission v Frucona Košice a,2,prem,D13,connection also borne mind lawfulness decision...,L,Prec,"[0.83251953125, -0.8701171875, -0.16796875, 1...."
2124,R2017_European Commission v Frucona Košice a,2,prem,D14,However information ‘available’ Commission inc...,L,"[Itpr, Prec]","[1.4931640625, -1.1728515625, -0.70751953125, ..."
2126,R2017_European Commission v Frucona Košice a,2,prem,D16,Paragraphs 180 213 235 judgment appeal Commiss...,F,Itpr,"[0.55419921875, -1.5498046875, -0.5234375, 1.4..."
2130,R2017_European Commission v Frucona Košice a,2,prem,D20,Advocate General noted paragraphs 125 131 Opin...,F,Aut,"[1.2119140625, -1.08984375, 0.0924072265625, -..."


In [6]:
Training_graph = nx.Graph() #Resoluti

for i in range(len(df_train)):
    print('adding node',i)
    Training_graph.add_node(i)
    for j in range(i):
        try:
            Training_graph.add_edge(i,j, weight = 1/euclid(df_train.iloc[j]['embeddings'], df_train.iloc[i]['embeddings']))
        except: 
            Training_graph.add_edge(i,j, weight = 1)

adding node 0
adding node 1
adding node 2
adding node 3
adding node 4
adding node 5
adding node 6
adding node 7
adding node 8
adding node 9
adding node 10
adding node 11
adding node 12
adding node 13
adding node 14
adding node 15
adding node 16
adding node 17
adding node 18
adding node 19
adding node 20
adding node 21
adding node 22
adding node 23
adding node 24
adding node 25
adding node 26
adding node 27
adding node 28
adding node 29
adding node 30
adding node 31
adding node 32
adding node 33
adding node 34
adding node 35
adding node 36
adding node 37
adding node 38
adding node 39
adding node 40
adding node 41
adding node 42
adding node 43
adding node 44
adding node 45
adding node 46
adding node 47
adding node 48
adding node 49
adding node 50
adding node 51
adding node 52
adding node 53
adding node 54
adding node 55
adding node 56
adding node 57
adding node 58
adding node 59
adding node 60
adding node 61
adding node 62
adding node 63
adding node 64
adding node 65
adding node 66
addin

In [7]:
pickle.dump(Training_graph, open('R15_task3_weighted.pickle', 'wb'))


In [8]:
len(Training_graph)

779